In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets,transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,32,kernel_size=3,padding=1)
        self.conv2 = nn.Conv2d(32,64,kernel_size=3,padding=1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(64*14*14,128) 
        self.fc2 = nn.Linear(128,10)
    
    def forward(self,x):
        x = nn.functional.relu(self.conv1(x))
        x = nn.functional.relu(self.conv2(x))
        x = nn.functional.max_pool2d(self.dropout1(x),2)
        x = torch.flatten(x,1)
        x = nn.functional.relu(self.fc1(self.dropout2(x)))
        x = self.fc2(x)
        return nn.functional.log_softmax(x,dim=1)

In [4]:
device = torch.device("cuda"  if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,),(0.3081,))
])

train_dataset = datasets.MNIST('./data',
                               train=True,
                               download=True,
                               transform=transform
                               )
test_dataset = datasets.MNIST('./data',
                              train=False,
                              transform=transform
                              )

batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset,
                                           batch_size=batch_size,
                                           shuffle=True
                                           )
test_loader = torch.utils.data.DataLoader(test_dataset,
                                          batch_size=batch_size,
                                          shuffle=True
                                          )

model = Net().to(device)
optimizer = optim.Adam(model.parameters())

model.train()
for batch_idx,(data,target) in enumerate(train_loader):
    data,target = data.to(device),target.to(device)
    optimizer.zero_grad()
    output = model(data)
    loss = nn.functional.nll_loss(output,target)
    loss.backward()
    optimizer.step()

model.eval()
test_loss = 0
correct = 0
with torch.no_grad():
    for data,target in test_loader:
        data,target = data.to(device),target.to(device)
        output = model(data)

        test_loss += nn.functional.nll_loss(output, target, reduction='sum').item() 
        
        
        pred = output.argmax(dim=1,keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
        

In [5]:
test_loss

0.045554751210659744